### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [145]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

from IPython.core.magic import (register_line_magic,
                                register_cell_magic)

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [146]:
@register_line_magic
def xtime(line):
    t = time()
    r = get_ipython().ev(line)
    t = time() - t
    print(f'Done in {t:.3f} sec: {line}')
    return r

@register_cell_magic
def xtime(line, cell):
    t = time()
    get_ipython().run_cell(cell)
    t = time() - t
    if line:
        print(f'Done in {t:.3f} sec: {line}')
    else:
        print(f'Done in {t:.3f} sec')

@register_cell_magic
def xcodestyle(line, cell):
    ipython = get_ipython()
    ipython.run_cell_magic('pycodestyle', line, cell)
    ipython.run_cell(cell)

In [147]:
try:
    from numba import autojit
except:
    def autojit(func):
        return func

@autojit
def get_class_count(y, cut_size=0):
    classes = np.unique(y)
    l_c = np.zeros(shape=(classes.size, y.size), dtype=int)
    for i, c in enumerate(classes):
        l_c[i] = np.cumsum(y==c)
    r_c = l_c[:, -1:] - l_c[:, :-1]
    s = np.arange(1, y.size, dtype=int)
    
    l_c, l_s, r_c, r_s =  l_c[:, :-1], s, r_c, s[::-1]
        
    if cut_size:
        l_c = l_c[:, cut_size: -cut_size]
        l_s = l_s[cut_size: -cut_size]
        r_c = r_c[:, cut_size: -cut_size]
        r_s = r_s[cut_size: -cut_size]
    return l_c, l_s, r_c, r_s

@autojit
def get_class_count2(y, cut_size=0):
    if cut_size == 0:
        splitted = y
    else:
        splitted = y[cut_size:-cut_size]
    r_border_ids = np.where(splitted[:-1] !=
                            splitted[1:])[0] + (cut_size + 1)

    if len(r_border_ids) == 0:
        return call(), np.inf, None

    class_number = np.max(y) + 1
    eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                           r_border_ids[:-1])
    one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
    one_hot_code[np.arange(r_border_ids.shape[0]),
                 y[r_border_ids - 1]] = 1
    class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
    class_increments[0] = class_increments[0] + \
        np.bincount(y[:cut_size], minlength=class_number)

    l_c = np.cumsum(class_increments, axis=0)
    r_c = np.bincount(y, minlength=class_number) - l_c
    l_s = r_border_ids.reshape(l_c.shape[0], 1)
    r_s = y.shape[0] - l_s
    return l_c.T, l_s.flatten(), r_c.T, r_s.flatten()

def get_class_count_slow(y, cut_size=0):
    classes = np.unique(y)
    l_c = np.zeros(shape=(classes.size, y.size), dtype=int)
    for i, c in enumerate(classes):
        l_c[i] = np.cumsum(y==c)
    r_c = l_c[:, -1:] - l_c[:, :-1]
    s = np.arange(1, y.size, dtype=int)
    
    l_c, l_s, r_c, r_s =  l_c[:, :-1], s, r_c, s[::-1]
        
    if cut_n:
        l_c = l_c[:, cut_size: -cut_size]
        l_s = l_s[cut_size: -cut_size]
        r_c = r_c[:, cut_size: -cut_size]
        r_s = r_s[cut_size: -cut_size]
    return l_c, l_s, r_c, r_s

In [148]:
%%xcodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_function_y = self.__gini_y
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_function_y = self.__entropy_y
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_function_y = self.__misclass_y
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # Ваш код
        return 1 - np.sum(l_c**2/l_s + r_c**2/r_s, 0) / (l_s+r_s)

    def __entropy(self, l_c, l_s, r_c, r_s):
        # Ваш код
        return -np.sum(l_c * np.log2(l_c/l_s)
                       + r_c * np.log2(r_c/r_s), 0) / (l_s+r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        # Ваш код
        return 1 - (np.max(l_c, 0) + np.max(r_c, 0)) / (l_s+r_s)

    def __gini_y(self, y_c, y_s):
        return y_s - np.sum(y_c**2) / y_s

    def __entropy_y(self, y_c, y_s):
        return -np.sum(y_c * np.log2(y_c/y_s)) / (y_s)

    def __misclass_y(self, y_c, y_s):
        return y_s - np.max(y_c) / y_s

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        # Ваш код
        return feature_ids[:np.sqrt(n_feature)]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        # Ваш код
        return feature_ids[:np.log2(n_feature)]

    def __get_feature_ids_N(self, n_feature):
        # Ваш код
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __add_leaf(self, node_id, y):
        self.tree[node_id] = (self.LEAF_TYPE,
                              np.bincount(y).argmax(),
                              np.bincount(y) / y.size)
        return

    def __add_node(self, node_id, feature_id, threshold):
        self.tree[node_id] = (self.NON_LEAF_TYPE, feature_id, threshold)
        return

    def __find_threshold(self, x, y):
        # Ваш код
        x_sort, y_sort = self.__sort_samples(x, y)

        cut_n = self.min_samples_split // 2 - 1
        l_c, l_s, r_c, r_s = get_class_count2(y_sort, cut_n)

        G = self.G_function_y(l_c[:, 0] + r_c[:, 0], y.size)
        Gs = self.G_function(l_c, l_s, r_c, r_s)

        idx = np.argmin(Gs)
        x_idx = l_s[idx]

        res = Gs[idx], G, (x_sort[x_idx-1] + x_sort[x_idx])/2
        return res

    def __fit_node(self, x, y, node_id, depth):
        # Ваш код
        if ((self.max_depth is not None and self.max_depth < depth)
                or (y.size < self.min_samples_split)
                or (np.unique(y).size == 1)
                or (np.bincount(y).argmax() > y.size * self.sufficient_share)):
            self.__add_leaf(node_id, y)
            return

        feature_ids = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, feature_id], y)
                               for feature_id in feature_ids])

        best_feature = feature_ids[np.argmin(thresholds[:, 0])]
        neg_gain, pos_gain, threshold = thresholds[np.argmin(thresholds[:, 0])]

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_feature, threshold)

        if y_l.size == 0 or y_r.size == 0:
            self.__add_leaf(node_id, y)
            return

        self.__add_node(node_id, best_feature, threshold)
        self.feature_importances_[best_feature] += pos_gain - neg_gain

        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
        return

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= np.sum(self.feature_importances_)
        return self

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [167]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [168]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [175]:
%xtime clf.fit(X_train, y_train)

Done in 0.002 sec: clf.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [176]:
%xtime my_clf.fit(X_train, y_train)

Done in 0.034 sec: my_clf.fit(X_train, y_train)


## Проверка качества работы на wine

In [177]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.890993265993266

In [178]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.890993265993266

## Подготовка данных Speed Dating Data 

In [155]:
df = pd.read_csv('./speed-dating-experiment/Speed Dating Data.csv', encoding='latin1')
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
             'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'undergra',
              'from', 'zipcode', 'sports','tvsports','exercise','dining',
              'museums','art','hiking','gaming', 'clubbing','reading','tv',
              'theater','movies','concerts','music','shopping','yoga',
              'expnum'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(df.mn_sat[df.mn_sat.notna()].mean())

df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(df.tuition[df.tuition.notna()].mean())

df = df.dropna(subset=['imprelig', 'imprace'])
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

df = df.dropna(subset=['date'])

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)

df.loc[:, 'temp_totalsum'] = df.loc[:,
                                    ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                                     'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T
     / df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, 
                                    ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                                     'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T
     / df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Проверка скорости работы на Speed Dating Data 

In [163]:
%xtime clf.fit(X_train, y_train)

Done in 0.129 sec: clf.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [157]:
%xtime my_clf.fit(X_train, y_train)

Done in 1.434 sec: my_clf.fit(X_train, y_train)


## Проверка качества работы на Speed Dating Data

In [164]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5287952351069569

In [159]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5738425644967702

## Задание 3

In [160]:
features = df_pair.columns[1:]

In [161]:

pd.Series(data=clf.feature_importances_,
          index=features).sort_values(ascending=False).head(10)

int_corr      0.080906
age_f         0.027607
attr2_1       0.027086
age           0.025211
attr1_1_f     0.024801
shar1_1_f     0.024539
intel3_1_f    0.024479
exphappy_f    0.023664
amb3_1_f      0.023652
income_f      0.023235
dtype: float64

In [162]:
pd.Series(data=my_clf.feature_importances_,
          index=features).sort_values(ascending=False).head(10)

date         0.145050
shar1_1      0.135819
age          0.115927
attr3_1      0.104181
go_out       0.048375
amb1_1_f     0.047589
int_corr     0.041690
tuition_f    0.029992
fun2_1_f     0.028648
shar1_1_f    0.020429
dtype: float64

## Задание 4

In [179]:
%%xtime

rfc = RandomForestClassifier(n_jobs=-1, n_estimators=10)

param_grid = { 
    'max_depth': [1,30],
    'min_samples_leaf': range(1,30, 2),
    'criterion': ['gini', 'entropy']
}

CV = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV.fit(X, y)
print(CV.best_params_)
print(CV.best_score_)

{'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 15}
0.8367091772943236
Done in 101.945 sec
